In [1]:
import heapq

class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def _generate_codes_recursive(node, prefix, codes):
    if node.char is not None:
        codes[node.char] = prefix if prefix else '0'
        return

    if node.left:
        _generate_codes_recursive(node.left, prefix + '0', codes)
    if node.right:
        _generate_codes_recursive(node.right, prefix + '1', codes)

def huffman_encoding(frequency_distribution: dict):
    if not frequency_distribution:
        print("Input frequency distribution is empty.")
        return None

    priority_queue = [HuffmanNode(char, freq) for char, freq in frequency_distribution.items()]
    heapq.heapify(priority_queue)

    if len(priority_queue) == 1:
        root = priority_queue[0]
    else:
        while len(priority_queue) > 1:
            left_child = heapq.heappop(priority_queue)
            right_child = heapq.heappop(priority_queue)
            
            merged_freq = left_child.freq + right_child.freq
            merged_node = HuffmanNode(None, merged_freq)
            merged_node.left = left_child
            merged_node.right = right_child

            heapq.heappush(priority_queue, merged_node)
        
        root = priority_queue[0]

    codes = {}
    _generate_codes_recursive(root, "", codes)
    
    print("Huffman Prefix Codes:")
    for char in sorted(codes.keys()):
        print(f"  '{char}': {codes[char]}")

    return root

if __name__ == "__main__":
    print("--- Test Case 1: Provided Example ---")
    freq1 = {'A': 45, 'B': 13, 'C': 12, 'D': 16, 'E': 9, 'F': 5}
    huffman_tree_root1 = huffman_encoding(freq1)
    print("-" * 40)

    print("--- Test Case 2: Simple Alphabet ---")
    freq2 = {'X': 5, 'Y': 10, 'Z': 15, 'W': 20}
    huffman_tree_root2 = huffman_encoding(freq2)
    print("-" * 40)
    
    print("--- Test Case 3: Highly Skewed Frequencies ---")
    freq3 = {'T': 100, 'U': 5, 'V': 1}
    huffman_tree_root3 = huffman_encoding(freq3)
    print("-" * 40)

    print("--- Test Case 4: Balanced Frequencies ---")
    freq4 = {'P': 8, 'Q': 8, 'R': 8, 'S': 8}
    huffman_tree_root4 = huffman_encoding(freq4)
    print("-" * 40)

    print("--- Test Case 5: Single Character (Edge Case) ---")
    freq5 = {'G': 50}
    huffman_tree_root5 = huffman_encoding(freq5)
    print("-" * 40)

--- Test Case 1: Provided Example ---
Huffman Prefix Codes:
  'A': 0
  'B': 101
  'C': 100
  'D': 111
  'E': 1101
  'F': 1100
----------------------------------------
--- Test Case 2: Simple Alphabet ---
Huffman Prefix Codes:
  'W': 0
  'X': 110
  'Y': 111
  'Z': 10
----------------------------------------
--- Test Case 3: Highly Skewed Frequencies ---
Huffman Prefix Codes:
  'T': 1
  'U': 01
  'V': 00
----------------------------------------
--- Test Case 4: Balanced Frequencies ---
Huffman Prefix Codes:
  'P': 01
  'Q': 11
  'R': 00
  'S': 10
----------------------------------------
--- Test Case 5: Single Character (Edge Case) ---
Huffman Prefix Codes:
  'G': 0
----------------------------------------


In [3]:
import heapq
import time
import random
import sys

infinity = float('inf')

def create_graph_from_edges(edges, num_vertices):
    adj_list = {i: [] for i in range(num_vertices)}
    for u, v, weight in edges:
        adj_list[u].append((v, weight))
        adj_list[v].append((u, weight))
    return adj_list

def generate_large_random_graph(num_vertices, num_edges):
    if num_edges < num_vertices - 1:
        raise ValueError("Number of edges must be at least num_vertices - 1 for a connected graph.")

    edges = set()
    
    vertices = list(range(num_vertices))
    random.shuffle(vertices)
    for i in range(num_vertices - 1):
        weight = random.randint(1, 100)
        u, v = vertices[i], vertices[i+1]
        edges.add(tuple(sorted((u, v))) + (weight,))

    while len(edges) < num_edges:
        u, v = random.sample(range(num_vertices), 2)
        if u != v:
            weight = random.randint(1, 100)
            edges.add(tuple(sorted((u, v))) + (weight,))
            
    return create_graph_from_edges(list(edges), num_vertices), num_vertices

def prims_naive(graph, num_vertices, start_vertex=0):
    min_weight = {i: infinity for i in range(num_vertices)}
    visited = set()
    mst_cost = 0
    min_weight[start_vertex] = 0

    for _ in range(num_vertices):
        min_w = infinity
        u = -1
        for v_idx in range(num_vertices):
            if v_idx not in visited and min_weight[v_idx] < min_w:
                min_w = min_weight[v_idx]
                u = v_idx

        if u == -1:
            break

        visited.add(u)
        mst_cost += min_weight[u]

        for neighbor, weight in graph.get(u, []):
            if neighbor not in visited and weight < min_weight[neighbor]:
                min_weight[neighbor] = weight
                
    return mst_cost

def prims_optimized(graph, num_vertices, start_vertex=0):
    priority_queue = [(0, start_vertex)]
    visited = set()
    mst_cost = 0

    while priority_queue and len(visited) < num_vertices:
        weight, u = heapq.heappop(priority_queue)

        if u in visited:
            continue

        visited.add(u)
        mst_cost += weight
        
        for neighbor, edge_weight in graph.get(u, []):
            if neighbor not in visited:
                heapq.heappush(priority_queue, (edge_weight, neighbor))
                
    return mst_cost

if __name__ == "__main__":
    NUM_VERTICES = 1000
    NUM_EDGES = 15000
    
    print(f"Generating a large random graph with {NUM_VERTICES} vertices and {NUM_EDGES} edges...")
    large_graph, num_v = generate_large_random_graph(NUM_VERTICES, NUM_EDGES)
    print("Graph generation complete.")
    print("-" * 50)

    print("Running Un-optimized Prim's Algorithm...")
    start_time_naive = time.time()
    mst_cost_naive = prims_naive(large_graph, num_v)
    end_time_naive = time.time()
    duration_naive = end_time_naive - start_time_naive
    print(f"MST Cost (Naive): {mst_cost_naive}")
    print(f"Execution Time: {duration_naive:.4f} seconds")
    print("-" * 50)
    
    print("Running Optimized Prim's Algorithm...")
    start_time_optimized = time.time()
    mst_cost_optimized = prims_optimized(large_graph, num_v)
    end_time_optimized = time.time()
    duration_optimized = end_time_optimized - start_time_optimized
    print(f"MST Cost (Optimized): {mst_cost_optimized}")
    print(f"Execution Time: {duration_optimized:.4f} seconds")
    print("-" * 50)
    
    print("\n--- Performance Comparison ---")
    if mst_cost_naive == mst_cost_optimized:
        print("✅ Both algorithms correctly computed the same MST cost.")
    else:
        print("❌ Error: MST costs do not match!")
        
    if duration_optimized < duration_naive:
        speedup = duration_naive / duration_optimized
        print(f"🚀 The optimized version was {speedup:.2f} times faster than the naive version.")
    else:
        print("The optimized version was not faster. This might happen on very small graphs.")

Generating a large random graph with 1000 vertices and 15000 edges...
Graph generation complete.
--------------------------------------------------
Running Un-optimized Prim's Algorithm...
MST Cost (Naive): 4676
Execution Time: 0.0708 seconds
--------------------------------------------------
Running Optimized Prim's Algorithm...
MST Cost (Optimized): 4676
Execution Time: 0.0079 seconds
--------------------------------------------------

--- Performance Comparison ---
✅ Both algorithms correctly computed the same MST cost.
🚀 The optimized version was 8.93 times faster than the naive version.


In [4]:
import sys

def greedy_coin_change(denominations, amount):
    sorted_denominations = sorted(denominations, reverse=True)
    
    result_coins = []
    remaining_amount = amount
    
    for coin in sorted_denominations:
        while remaining_amount >= coin:
            result_coins.append(coin)
            remaining_amount -= coin
            
    if remaining_amount == 0:
        return result_coins
    else:
        return -1

def dp_coin_change(denominations, amount):
    dp = [float('inf')] * (amount + 1)
    coin_used = [0] * (amount + 1)
    dp[0] = 0
    
    for i in range(1, amount + 1):
        for coin in denominations:
            if coin <= i:
                if 1 + dp[i - coin] < dp[i]:
                    dp[i] = 1 + dp[i - coin]
                    coin_used[i] = coin

    if dp[amount] == float('inf'):
        return -1
        
    result_coins = []
    current_amount = amount
    while current_amount > 0:
        coin = coin_used[current_amount]
        result_coins.append(coin)
        current_amount -= coin
        
    return result_coins

if __name__ == "__main__":
    
    print("--- Test Case 1: Canonical Coin System ---")
    denominations1 = [1, 5, 10, 25]
    amount1 = 99
    print(f"Denominations: {denominations1}, Amount: {amount1}")
    print(f"Greedy Solution:    {greedy_coin_change(denominations1, amount1)}")
    print(f"DP (Optimal) Solution: {dp_coin_change(denominations1, amount1)}")
    print("-" * 50)

    print("--- Test Case 2: Non-Canonical System (Greedy Fails) ---")
    denominations2 = [1, 3, 4]
    amount2 = 6
    print(f"Denominations: {denominations2}, Amount: {amount2}")
    print(f"Greedy Solution:    {greedy_coin_change(denominations2, amount2)}")
    print(f"DP (Optimal) Solution: {dp_coin_change(denominations2, amount2)}")
    print("-" * 50)
    
    print("--- Test Case 3: Another Non-Canonical System (Greedy Fails) ---")
    denominations3 = [1, 5, 10, 12]
    amount3 = 16
    print(f"Denominations: {denominations3}, Amount: {amount3}")
    print(f"Greedy Solution:    {greedy_coin_change(denominations3, amount3)}")
    print(f"DP (Optimal) Solution: {dp_coin_change(denominations3, amount3)}")
    print("-" * 50)
    
    print("--- Test Case 4: Impossible to Make Change ---")
    denominations4 = [5, 10]
    amount4 = 17
    print(f"Denominations: {denominations4}, Amount: {amount4}")
    print(f"Greedy Solution:    {greedy_coin_change(denominations4, amount4)}")
    print(f"DP (Optimal) Solution: {dp_coin_change(denominations4, amount4)}")
    print("-" * 50)

--- Test Case 1: Canonical Coin System ---
Denominations: [1, 5, 10, 25], Amount: 99
Greedy Solution:    [25, 25, 25, 10, 10, 1, 1, 1, 1]
DP (Optimal) Solution: [1, 1, 1, 1, 10, 10, 25, 25, 25]
--------------------------------------------------
--- Test Case 2: Non-Canonical System (Greedy Fails) ---
Denominations: [1, 3, 4], Amount: 6
Greedy Solution:    [4, 1, 1]
DP (Optimal) Solution: [3, 3]
--------------------------------------------------
--- Test Case 3: Another Non-Canonical System (Greedy Fails) ---
Denominations: [1, 5, 10, 12], Amount: 16
Greedy Solution:    [12, 1, 1, 1, 1]
DP (Optimal) Solution: [1, 5, 10]
--------------------------------------------------
--- Test Case 4: Impossible to Make Change ---
Denominations: [5, 10], Amount: 17
Greedy Solution:    -1
DP (Optimal) Solution: -1
--------------------------------------------------
